In [1]:
import zarr
import xarray as xr
import time
import dask
import glob

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
c = Client(cluster)

c


Client Scheduler: tcp://127.0.0.1:40816 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 40 Memory: 197.57 GB


In [3]:
files=sorted(glob.glob('/work/ALT/odatis/eNATL60/BLBT02/gridT-2D/eNATL60*gridT-2D*.nc'))
drop_vars = ['nav_lat', 'nav_lon', 'somxl010', 'sosaline', 'sosstsst']
extra_coord_vars = []
chunks = dict(time_counter=240, y=240, x=480)
open_kwargs = dict(drop_variables=(drop_vars + extra_coord_vars),
                   chunks=chunks,decode_cf=True,decode_times=False,
                   concat_dim="time_counter",combine='nested')

In [4]:
%time ds = xr.open_mfdataset(files, parallel=True, **open_kwargs)

CPU times: user 3.5 s, sys: 259 ms, total: 3.75 s
Wall time: 19.8 s


In [5]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 1.11 s, sys: 27.5 ms, total: 1.13 s
Wall time: 1.13 s


In [ ]:
%time mean.load()

In [ ]:
cluster.close()